# 🔍 Interprétabilité des Modèles COVID-19 avec SHAP, GradCAM et LIME

Ce notebook démontre l'utilisation des techniques d'interprétabilité **SHAP**, **GradCAM** et **LIME** pour expliquer les prédictions des modèles de classification COVID-19.

## 📋 Objectifs

1. **🎯 GradCAM** : Visualiser les zones importantes pour les prédictions CNN
2. **📊 SHAP** : Quantifier l'importance des features pour tous types de modèles
3. **🍃 LIME** : Explications locales par superpixels pour modèles image
4. **⚖️ Comparaison** : Analyser la cohérence entre les différentes méthodes
5. **🏥 Application** : Cas d'usage médical pour le diagnostic COVID-19

## 🚀 Framework RAF

Nous utilisons le **framework RAF** (Raw Augmentation Framework) qui intègre maintenant les modules d'interprétabilité :
- `raf.interpretability.SHAPExplainer` - Explications SHAP universelles
- `raf.interpretability.GradCAMExplainer` - Visualisations GradCAM pour CNN
- `raf.interpretability.LIMEExplainer` - Explications LIME par superpixels
- `raf.interpretability.InterpretabilityAnalyzer` - Analyse comparative
- `raf.data.DataLoader` - Chargement de vos vraies données COVID-19

## ⚙️ Configuration et Imports

In [2]:
# Configuration universelle avec le framework RAF
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

# Chargement des variables d'environnement
print("🔧 Chargement de la configuration .env...")

# Détection automatique du répertoire racine
current_dir = Path().cwd()
if 'notebooks' in str(current_dir):
    project_root = current_dir.parent
else:
    project_root = current_dir

# Chargement du fichier .env
env_path = project_root / '.env'
if env_path.exists():
    load_dotenv(env_path)
    print(f"✅ Configuration chargée depuis: {env_path}")
else:
    print(f"⚠️ Fichier .env non trouvé: {env_path}")

# Configuration du PYTHONPATH avec les variables d'environnement
src_root = os.getenv('PROJECT_ROOT', str(project_root))
if src_root not in sys.path:
    sys.path.insert(0, src_root)

print(f"📁 Répertoire de travail: {project_root}")
print(f"🎯 Répertoire source: {src_root}")
print("🔧 Configuration du framework RAF...")

# Imports standards
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration graphiques depuis .env
plot_style = os.getenv('PLOT_STYLE', 'default')
color_palette = os.getenv('COLOR_PALETTE', 'husl')

try:
    plt.style.use(plot_style)
except:
    plt.style.use('default')
    
sns.set_palette(color_palette)
%matplotlib inline

# Configuration warnings
import warnings
warnings.filterwarnings('ignore')

print("✅ Configuration de base terminée")

🔧 Chargement de la configuration .env...
✅ Configuration chargée depuis: /home/cepa/DST/projet_DS/DS_COVID/.env
📁 Répertoire de travail: /home/cepa/DST/projet_DS/DS_COVID
🎯 Répertoire source: /home/cepa/DST/projet_DS/DS_COVID
🔧 Configuration du framework RAF...
✅ Configuration de base terminée
✅ Configuration de base terminée


In [3]:
# Imports du framework RAF
# Configuration du chemin d'import
import sys
import os
from pathlib import Path

# Détection automatique du répertoire racine
current_dir = Path.cwd()
if 'notebooks' in str(current_dir):
    project_root = current_dir.parent
else:
    project_root = current_dir

# Ajout du répertoire racine au PYTHONPATH
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"📁 Répertoire projet: {project_root}")
print(f"🐍 PYTHONPATH ajusté")

# Import direct depuis src/
try:
    from src.features.raf.interpretability import (
        SHAPExplainer,
        GradCAMExplainer,
        LIMEExplainer,
        InterpretabilityAnalyzer,
        create_interpretability_dashboard,
        generate_interpretability_report,
        visualize_lime_comparison
    )
    print("✅ Modules RAF d'interprétabilité chargés (SHAP, GradCAM, LIME)")
    RAF_AVAILABLE = True
except ImportError as e:
    print(f"❌ Erreur import RAF: {e}")
    print("💡 Vérifiez l'installation du framework RAF")
    RAF_AVAILABLE = False

# Import du DataLoader RAF pour les vraies données
try:
    from src.features.raf.data import DataLoader
    print("✅ DataLoader RAF chargé")
    DATALOADER_AVAILABLE = True
except ImportError as e:
    print(f"❌ Erreur DataLoader: {e}")
    DATALOADER_AVAILABLE = False

# Vérification des dépendances
dependencies = {
    'tensorflow': 'TensorFlow',
    'shap': 'SHAP',
    'cv2': 'OpenCV',
    'sklearn': 'Scikit-learn',
    'lime': 'LIME'
}

missing_deps = []
for module, name in dependencies.items():
    try:
        __import__(module)
        print(f"✅ {name} disponible")
    except ImportError:
        print(f"❌ {name} manquant")
        missing_deps.append(name)

if missing_deps:
    print(f"\n📦 Installation nécessaire:")
    for dep in missing_deps:
        if dep == 'SHAP':
            print("!pip install shap")
        elif dep == 'TensorFlow':
            print("!pip install tensorflow")
        elif dep == 'OpenCV':
            print("!pip install opencv-python")
        elif dep == 'LIME':
            print("!pip install lime")
else:
    print("\n🎉 Toutes les dépendances sont disponibles !")

print(f"\n📊 Statut des modules:")
print(f"• RAF Interpretability: {'✅' if RAF_AVAILABLE else '❌'}")
print(f"• DataLoader RAF: {'✅' if DATALOADER_AVAILABLE else '❌'}")

if RAF_AVAILABLE:
    print("🎉 Framework d'interprétabilité prêt avec SHAP, GradCAM et LIME !")

📁 Répertoire projet: /home/cepa/DST/projet_DS/DS_COVID
🐍 PYTHONPATH ajusté


2025-10-16 22:16:40.789604: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


🎨 RAF (Raw Augmentation Framework) v2.0.0 chargé avec succès
✨ NOUVELLE FONCTIONNALITÉ: setup_universal_environment() remplace la cellule 1!
� NOUVEAU MODULE: interpretability (SHAP + GradCAM)
��������� Modules disponibles: utils, data, augmentation, interpretability
✅ Modules RAF d'interprétabilité chargés (SHAP, GradCAM, LIME)
✅ DataLoader RAF chargé
✅ TensorFlow disponible
✅ SHAP disponible
✅ OpenCV disponible
✅ Scikit-learn disponible
✅ LIME disponible

🎉 Toutes les dépendances sont disponibles !

📊 Statut des modules:
• RAF Interpretability: ✅
• DataLoader RAF: ✅
🎉 Framework d'interprétabilité prêt avec SHAP, GradCAM et LIME !


## 📊 Chargement des Données et Modèles

Nous allons utiliser des données simulées pour la démonstration. En pratique, vous remplacerez ceci par vos vrais modèles et données.

In [ ]:
# Chargement des vraies données COVID-19 avec DataLoader RAF
print("🔄 Chargement des données réelles COVID-19...")

# Import de tqdm pour les barres de progression
from tqdm.notebook import tqdm
import time

if DATALOADER_AVAILABLE:
    # 🎯 Configuration depuis .env avec CHEMINS ABSOLUS (Plus de problèmes !)
    print("⚙️ Chargement de la configuration depuis .env...")
    
    # Chemins absolus depuis .env
    DATA_DIR = os.getenv('DATA_DIR', '/home/cepa/DST/projet_DS/DS_COVID/data/raw/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset')
    MODELS_DIR = os.getenv('MODELS_DIR', '/home/cepa/DST/projet_DS/DS_COVID/models')
    RESULTS_DIR = os.getenv('RESULTS_DIR', '/home/cepa/DST/projet_DS/DS_COVID/reports')
    PROJECT_ROOT = os.getenv('PROJECT_ROOT', '/home/cepa/DST/projet_DS/DS_COVID')
    
    # Paramètres d'images
    IMG_WIDTH = int(os.getenv('IMG_WIDTH', 224))
    IMG_HEIGHT = int(os.getenv('IMG_HEIGHT', 224))
    IMG_SIZE = (IMG_WIDTH, IMG_HEIGHT)
    
    # Paramètres de classes
    NUM_CLASSES = int(os.getenv('NUM_CLASSES', 4))
    CLASS_NAMES = os.getenv('CLASS_NAMES', 'COVID,Lung_Opacity,Normal,Viral Pneumonia').split(',')
    RANDOM_SEED = int(os.getenv('RANDOM_SEED', 42))
    
    # Configuration d'interprétabilité depuis .env
    GRADCAM_ALPHA = float(os.getenv('GRADCAM_ALPHA', 0.4))
    GRADCAM_COLORMAP = os.getenv('GRADCAM_COLORMAP', 'jet')
    SHAP_MAX_EVALS = int(os.getenv('SHAP_MAX_EVALS', 100))
    SHAP_BACKGROUND_SIZE = int(os.getenv('SHAP_BACKGROUND_SIZE', 50))
    LIME_NUM_SAMPLES = int(os.getenv('LIME_NUM_SAMPLES', 1000))
    
    print(f"\n📋 Configuration chargée (CHEMINS ABSOLUS):")
    print(f"🏠 Racine du projet: {PROJECT_ROOT}")
    print(f"📁 Répertoire de données: {DATA_DIR}")
    print(f"🤖 Répertoire modèles: {MODELS_DIR}")
    print(f"? Répertoire résultats: {RESULTS_DIR}")
    print(f"?🖼️ Taille d'image: {IMG_SIZE}")
    print(f"🏷️ Nombre de classes: {NUM_CLASSES}")
    print(f"📋 Classes: {CLASS_NAMES}")
    print(f"🎯 Config interprétabilité:")
    print(f"  • GradCAM Alpha: {GRADCAM_ALPHA}, Colormap: {GRADCAM_COLORMAP}")
    print(f"  • SHAP Max Evals: {SHAP_MAX_EVALS}, Background Size: {SHAP_BACKGROUND_SIZE}")
    print(f"  • LIME Samples: {LIME_NUM_SAMPLES}")
    
    # Vérification simple de la structure avec chemins absolus
    from pathlib import Path
    data_path = Path(DATA_DIR)
    
    print(f"\n🔍 Vérification de la structure des données...")
    with tqdm(total=len(CLASS_NAMES) + 2, desc="Vérification structure", colour='blue') as pbar:
        print(f"📂 Chemin absolu: {data_path}")
        print(f"📂 Existe: {data_path.exists()}")
        pbar.update(1)
        
        if data_path.exists():
            # Vérifier chaque classe
            found_classes = []
            class_counts_manual = {}
            
            for class_name in CLASS_NAMES:
                class_path = data_path / class_name
                exists = class_path.exists()
                
                if exists:
                    # Compter les images
                    images = list(class_path.glob('**/*.png')) + list(class_path.glob('**/*.jpg')) + list(class_path.glob('**/*.jpeg'))
                    image_count = len(images)
                    class_counts_manual[class_name] = image_count
                    
                    print(f"📁 {class_name}: ✅ {image_count} images")
                    if image_count > 0:
                        found_classes.append(class_name)
                else:
                    print(f"📁 {class_name}: ❌ (dossier introuvable)")
                    class_counts_manual[class_name] = 0
                
                pbar.update(1)
            
            total_images_manual = sum(class_counts_manual.values())
            print(f"\n✅ Résumé:")
            print(f"📊 Classes trouvées: {len(found_classes)}/{len(CLASS_NAMES)}")
            print(f"🖼️ Total images: {total_images_manual}")
            pbar.update(1)
            
        else:
            print("❌ Le répertoire de données n'existe pas !")
            pbar.update(len(CLASS_NAMES) + 1)
    
    # Initialisation du DataLoader
    try:
        print("\n🔧 Configuration du DataLoader...")
        with tqdm(total=4, desc="Configuration", colour='green') as pbar:
            from src.features.raf.utils import get_config, Config
            config = get_config()
            pbar.update(1)
            
            # Configuration avec chemins absolus
            config.data_dir = data_path
            config.img_size = IMG_SIZE
            config.classes = CLASS_NAMES
            config.random_seed = RANDOM_SEED
            config.max_images_per_class = int(os.getenv('MAX_IMAGES_PER_CLASS', 100))
            pbar.update(1)
            
            print(f"🔧 Configuration DataLoader:")
            print(f"  • Data dir (absolu): {config.data_dir}")
            print(f"  • Classes: {config.classes}")
            print(f"  • Max images/classe: {config.max_images_per_class}")
            pbar.update(1)
            
            # Créer le DataLoader avec chemin absolu
            loader = DataLoader(data_dir=data_path, config=config)
            pbar.update(1)
        
        print(f"✅ DataLoader configuré avec succès")
        
        print(f"\n📊 Chargement du dataset...")
        
        # Charger les chemins et labels avec progression
        with tqdm(total=1, desc="Scan des fichiers", colour='orange') as pbar:
            try:
                print("🔍 Lancement du scan...")
                image_paths, labels, class_counts = loader.load_image_paths_and_labels()
                pbar.update(1)
                print(f"✅ Scan terminé: {len(image_paths)} images trouvées")
                
                if class_counts:
                    print(f"📊 Répartition détaillée par classe:")
                    for class_name, count in class_counts.items():
                        print(f"  • {class_name}: {count} images")
                else:
                    print("⚠️ Aucun comptage de classes disponible")
                    
            except Exception as e:
                print(f"❌ Erreur lors du scan: {e}")
                print(f"🔍 Détails de l'erreur:")
                import traceback
                traceback.print_exc()
                pbar.update(1)
                image_paths, labels, class_counts = [], [], {}
        
        if len(image_paths) > 0:
            # Créer un sous-ensemble équilibré pour la démo
            max_per_class = min(20, max(class_counts.values()) if class_counts.values() else 10)
            
            with tqdm(total=1, desc="Création sous-ensemble", colour='purple') as pbar:
                subset_paths, subset_labels = loader.create_balanced_subset(
                    image_paths, labels, max_per_class=max_per_class
                )
                pbar.update(1)
            
            print(f"📦 Sous-ensemble créé: {len(subset_paths)} images ({max_per_class} par classe)")
            
            # Charger un échantillon d'images pour visualisation
            n_samples = min(10, len(subset_paths))
            
            print(f"\n🖼️ Chargement de {n_samples} images échantillons...")
            with tqdm(total=1, desc="Chargement images", colour='cyan') as pbar:
                try:
                    sample_images, sample_labels_final = loader.load_sample_images(
                        subset_paths, subset_labels, n_samples=n_samples
                    )
                    pbar.update(1)
                    print(f"✅ {len(sample_images)} images chargées avec succès")
                except Exception as e:
                    print(f"❌ Erreur chargement images: {e}")
                    sample_images, sample_labels_final = [], []
                    pbar.update(1)
            
            if len(sample_images) > 0:
                # Convertir en format numpy pour les modèles
                print("\n⚙️ Conversion des données...")
                with tqdm(total=3, desc="Conversion numpy", colour='yellow') as pbar:
                    X_images = np.array(sample_images)
                    pbar.update(1)
                    
                    if len(X_images.shape) == 3:
                        X_images = np.expand_dims(X_images, axis=-1)
                    pbar.update(1)
                    
                    # Convertir les labels en indices
                    label_to_idx = {label: idx for idx, label in enumerate(CLASS_NAMES)}
                    y_true = np.array([label_to_idx.get(label, 0) for label in sample_labels_final])
                    pbar.update(1)
                
                # Version aplatie pour les modèles ML classiques
                with tqdm(total=1, desc="Aplatissement ML", colour='magenta') as pbar:
                    X_flat = X_images.reshape(len(X_images), -1)
                    pbar.update(1)
                
                print(f"\n📊 Données chargées pour la démo:")
                print(f"• Images CNN: {X_images.shape}")
                print(f"• Images ML: {X_flat.shape}")
                print(f"• Labels: {y_true.shape}")
                print(f"• Classes uniques: {np.unique(y_true)}")
                print(f"• Labels échantillons: {sample_labels_final}")
                
                # Affichage de quelques échantillons
                print("\n🎨 Affichage des échantillons...")
                fig, axes = plt.subplots(2, 5, figsize=(15, 6))
                
                with tqdm(total=min(10, len(X_images)), desc="Affichage", colour='red') as pbar:
                    for i in range(min(10, len(X_images))):
                        row, col = i // 5, i % 5
                        axes[row, col].imshow(X_images[i].squeeze(), cmap='gray')
                        axes[row, col].set_title(f'{sample_labels_final[i] if i < len(sample_labels_final) else "N/A"}')
                        axes[row, col].axis('off')
                        pbar.update(1)
                
                plt.suptitle('Échantillons de Données Réelles COVID-19', fontsize=14)
                plt.tight_layout()
                plt.show()
                
                REAL_DATA_LOADED = True
                
                print(f"\n🎉 SUCCÈS AVEC CHEMINS ABSOLUS !")
                print(f"📊 {len(image_paths)} images totales trouvées")
                print(f"🎯 {len(sample_images)} images échantillons prêtes pour l'interprétabilité")
                print(f"✨ Prêt pour SHAP, GradCAM et LIME !")
                
            else:
                print("⚠️ Aucune image n'a pu être chargée")
                REAL_DATA_LOADED = False
                
        else:
            print("⚠️ Aucune image trouvée dans le dataset")
            REAL_DATA_LOADED = False
            
    except Exception as e:
        print(f"❌ Erreur chargement dataset: {e}")
        print(f"💡 Vérifiez que le dossier {DATA_DIR} existe et contient les données")
        print(f"🔍 Détails de l'erreur:")
        import traceback
        traceback.print_exc()
        REAL_DATA_LOADED = False
        
else:
    print("❌ DataLoader RAF non disponible")
    print("💡 Vérifiez l'installation des dépendances RAF")
    REAL_DATA_LOADED = False

print(f"\n📋 STATUT FINAL:")
print(f"🔧 DataLoader disponible: {'✅' if DATALOADER_AVAILABLE else '❌'}")
print(f"📊 Données réelles chargées: {'✅' if REAL_DATA_LOADED else '❌'}")
print(f"🎯 Configuration: CHEMINS ABSOLUS (Fini les galères !)")
if REAL_DATA_LOADED:
    print(f"🚀 Prêt pour la suite : Modèles et Interprétabilité ! 🎉")

In [4]:
# 🧪 Test indépendant du DataLoader RAF
print("🧪 TEST INDÉPENDANT DU DATALOADER RAF")
print("=" * 50)

# Configuration du test
from pathlib import Path
import os
from tqdm.notebook import tqdm

# Configuration de base
DATA_DIR = os.getenv('DATA_DIR', './data/raw/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset')
CLASS_NAMES = ['COVID', 'Lung_Opacity', 'Normal', 'Viral Pneumonia']

print(f"📂 Répertoire de test: {DATA_DIR}")
print(f"🏷️ Classes à tester: {CLASS_NAMES}")

# Test 1: Vérification directe des dossiers
print(f"\n🔍 TEST 1: Vérification directe des dossiers")
print("-" * 40)

data_path = Path(DATA_DIR)
print(f"📍 Chemin absolu: {data_path.resolve()}")
print(f"📂 Existe: {data_path.exists()}")

if data_path.exists():
    all_items = list(data_path.iterdir())
    dirs_only = [p for p in all_items if p.is_dir()]
    
    print(f"📁 Tous les éléments ({len(all_items)}): {[p.name for p in all_items]}")
    print(f"📁 Dossiers seulement ({len(dirs_only)}): {[p.name for p in dirs_only]}")
    
    # Test de chaque classe
    print(f"\n🧪 Test de chaque classe:")
    for class_name in CLASS_NAMES:
        class_path = data_path / class_name
        exists = class_path.exists()
        is_dir = class_path.is_dir() if exists else False
        
        if exists and is_dir:
            # Compter les images
            images = list(class_path.glob('**/*.png')) + list(class_path.glob('**/*.jpg'))
            subfolders = [p for p in class_path.iterdir() if p.is_dir()]
            
            print(f"  ✅ {class_name}: {len(images)} images, {len(subfolders)} sous-dossiers")
            if subfolders:
                print(f"     📁 Sous-dossiers: {[p.name for p in subfolders]}")
        else:
            print(f"  ❌ {class_name}: {'fichier' if exists else 'inexistant'}")

# Test 2: Test du DataLoader si disponible
print(f"\n🔍 TEST 2: Test du DataLoader RAF")
print("-" * 40)

if DATALOADER_AVAILABLE:
    try:
        # Import et configuration
        from src.features.raf.data import DataLoader
        from src.features.raf.utils import get_config
        
        print("✅ Imports DataLoader réussis")
        
        # Configuration minimale
        config = get_config()
        config.data_dir = data_path
        config.classes = CLASS_NAMES
        config.max_images_per_class = 10  # Limité pour le test
        
        print(f"🔧 Configuration créée:")
        print(f"  • Data dir: {config.data_dir}")
        print(f"  • Classes: {config.classes}")
        
        # Initialisation du DataLoader
        print(f"\n🏗️ Création du DataLoader...")
        loader = DataLoader(data_dir=data_path, config=config)
        print("✅ DataLoader créé avec succès")
        
        # Test de la méthode load_image_paths_and_labels
        print(f"\n📊 Test de load_image_paths_and_labels()...")
        with tqdm(total=1, desc="Test chargement", colour='green') as pbar:
            try:
                image_paths, labels, class_counts = loader.load_image_paths_and_labels()
                pbar.update(1)
                
                print(f"✅ Méthode exécutée avec succès")
                print(f"📊 Résultats:")
                print(f"  • Images trouvées: {len(image_paths)}")
                print(f"  • Labels: {len(labels)}")
                print(f"  • Comptage par classe: {dict(class_counts) if class_counts else 'Vide'}")
                
                if len(image_paths) > 0:
                    print(f"  • Premier chemin: {image_paths[0]}")
                    print(f"  • Premier label: {labels[0]}")
                    
                    # Test de quelques échantillons
                    print(f"\n🖼️ Test de chargement d'images...")
                    try:
                        sample_images, sample_labels = loader.load_sample_images(
                            image_paths[:3], labels[:3], n_samples=3
                        )
                        print(f"✅ Échantillons chargés: {len(sample_images)} images")
                        if len(sample_images) > 0:
                            print(f"  • Shape première image: {sample_images[0].shape}")
                    except Exception as e:
                        print(f"❌ Erreur chargement échantillons: {e}")
                        
                else:
                    print(f"⚠️ Aucune image trouvée par le DataLoader")
                    
            except Exception as e:
                print(f"❌ Erreur dans load_image_paths_and_labels: {e}")
                import traceback
                traceback.print_exc()
                pbar.update(1)
        
    except Exception as e:
        print(f"❌ Erreur lors du test DataLoader: {e}")
        import traceback
        traceback.print_exc()
else:
    print("❌ DataLoader non disponible pour le test")

# Test 3: Test manuel de recherche d'images
print(f"\n🔍 TEST 3: Recherche manuelle d'images")
print("-" * 40)

manual_count = {}
total_manual = 0

for class_name in CLASS_NAMES:
    class_path = data_path / class_name
    if class_path.exists():
        # Recherche récursive d'images
        png_files = list(class_path.glob('**/*.png'))
        jpg_files = list(class_path.glob('**/*.jpg'))
        jpeg_files = list(class_path.glob('**/*.jpeg'))
        
        total_files = len(png_files) + len(jpg_files) + len(jpeg_files)
        manual_count[class_name] = total_files
        total_manual += total_files
        
        print(f"📁 {class_name}: {total_files} images (PNG: {len(png_files)}, JPG: {len(jpg_files)}, JPEG: {len(jpeg_files)})")
        
        if total_files > 0:
            first_file = (png_files + jpg_files + jpeg_files)[0]
            print(f"   📄 Premier fichier: {first_file.name}")
    else:
        manual_count[class_name] = 0
        print(f"📁 {class_name}: dossier inexistant")

print(f"\n📊 RÉSUMÉ DU TEST MANUEL:")
print(f"• Total images trouvées: {total_manual}")
print(f"• Répartition: {manual_count}")

# Conclusion du test
print(f"\n🎯 CONCLUSION DU TEST")
print("=" * 30)

if total_manual > 0:
    print("✅ Les images existent dans la structure de dossiers")
    if DATALOADER_AVAILABLE:
        print("🔧 Le problème semble venir du DataLoader RAF")
        print("💡 Suggestions:")
        print("  1. Vérifier la méthode load_image_paths_and_labels() du DataLoader")
        print("  2. Vérifier les patterns de recherche d'images")
        print("  3. Vérifier les chemins relatifs vs absolus")
    else:
        print("⚠️ DataLoader RAF non disponible")
else:
    print("❌ Aucune image trouvée manuellement")
    print("💡 Vérifiez la structure des dossiers et les chemins")

print(f"\n🏁 Test terminé - Vous pouvez maintenant identifier le problème !")

🧪 TEST INDÉPENDANT DU DATALOADER RAF
📂 Répertoire de test: /home/cepa/DST/projet_DS/DS_COVID/data/raw/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset
🏷️ Classes à tester: ['COVID', 'Lung_Opacity', 'Normal', 'Viral Pneumonia']

🔍 TEST 1: Vérification directe des dossiers
----------------------------------------
📍 Chemin absolu: /home/cepa/DST/projet_DS/DS_COVID/data/raw/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset
📂 Existe: True
📁 Tous les éléments (9): ['README.md.txt', 'Viral Pneumonia', 'Viral Pneumonia.metadata.xlsx', 'Normal.metadata.xlsx', 'Lung_Opacity.metadata.xlsx', 'Normal', 'COVID.metadata.xlsx', 'Lung_Opacity', 'COVID']
📁 Dossiers seulement (4): ['Viral Pneumonia', 'Normal', 'Lung_Opacity', 'COVID']

🧪 Test de chaque classe:
  ✅ COVID: 7232 images, 2 sous-dossiers
     📁 Sous-dossiers: ['images', 'masks']
  ✅ Lung_Opacity: 12024 images, 2 sous-dossiers
     📁 Sous-dossiers: ['images', 'masks']
  ✅ Normal: 20384 images, 2 sous-dossiers
     📁 Sous-dossi

Test chargement:   0%|          | 0/1 [00:00<?, ?it/s]

🔍 Chargement des chemins d'images...
  COVID: 3616 images
  Lung_Opacity: 6012 images
  COVID: 3616 images
  Lung_Opacity: 6012 images
  Normal: 10192 images
  Viral Pneumonia: 1345 images
📊 Total: 21165 images
✅ Méthode exécutée avec succès
📊 Résultats:
  • Images trouvées: 21165
  • Labels: 21165
  • Comptage par classe: {'COVID': 3616, 'Lung_Opacity': 6012, 'Normal': 10192, 'Viral Pneumonia': 1345}
  • Premier chemin: /home/cepa/DST/projet_DS/DS_COVID/data/raw/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset/COVID/images/COVID-3039.png
  • Premier label: COVID

🖼️ Test de chargement d'images...
🖼️ Chargement de 3 images d'exemple...
✅ 3 images chargées avec succès
✅ Échantillons chargés: 3 images
  • Shape première image: (256, 256, 3)

🔍 TEST 3: Recherche manuelle d'images
----------------------------------------
📁 COVID: 7232 images (PNG: 7232, JPG: 0, JPEG: 0)
   📄 Premier fichier: COVID-3039.png
📁 Lung_Opacity: 12024 images (PNG: 12024, JPG: 0, JPEG: 0)
   📄 Premier fic

In [12]:
# 🔧 CORRECTION AUTOMATIQUE DES CHEMINS
print("🔧 CORRECTION AUTOMATIQUE DES CHEMINS")
print("=" * 50)

from pathlib import Path
import os
from tqdm.notebook import tqdm

# Détection du problème de chemin
current_working_dir = Path.cwd()
print(f"📍 Répertoire de travail actuel: {current_working_dir}")

# Configuration des chemins possibles
possible_paths = [
    # Chemin depuis notebooks/ (actuel)
    Path('../data/raw/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset'),
    # Chemin depuis racine du projet
    Path('./data/raw/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset'),
    # Chemin avec un seul niveau
    Path('../data/raw/COVID-19_Radiography_Dataset'),
    # Chemin absolu basé sur project_root
    current_working_dir.parent / 'data' / 'raw' / 'COVID-19_Radiography_Dataset' / 'COVID-19_Radiography_Dataset'
]

CLASS_NAMES = ['COVID', 'Lung_Opacity', 'Normal', 'Viral Pneumonia']

print(f"\n🔍 Test des chemins possibles...")
correct_path = None

with tqdm(total=len(possible_paths), desc="Test chemins", colour='blue') as pbar:
    for i, test_path in enumerate(possible_paths):
        print(f"\n📂 Test {i+1}: {test_path}")
        print(f"   📍 Absolu: {test_path.resolve()}")
        print(f"   📂 Existe: {test_path.exists()}")
        
        if test_path.exists():
            # Vérifier la présence des dossiers de classes
            class_found = 0
            for class_name in CLASS_NAMES:
                class_path = test_path / class_name
                if class_path.exists():
                    class_found += 1
                    print(f"   ✅ {class_name}: {class_path.exists()}")
                else:
                    print(f"   ❌ {class_name}: {class_path.exists()}")
            
            if class_found == len(CLASS_NAMES):
                print(f"   🎯 CHEMIN CORRECT TROUVÉ !")
                correct_path = test_path
                break
            else:
                print(f"   ⚠️ Seulement {class_found}/{len(CLASS_NAMES)} classes trouvées")
        
        pbar.update(1)

# Résultat de la correction
if correct_path:
    print(f"\n✅ CHEMIN CORRIGÉ TROUVÉ !")
    print(f"📂 Chemin correct: {correct_path}")
    print(f"📍 Chemin absolu: {correct_path.resolve()}")
    
    # Mise à jour des variables d'environnement
    corrected_data_dir = str(correct_path)
    print(f"\n🔄 Mise à jour de la configuration...")
    
    # Test avec le bon chemin
    print(f"\n🧪 TEST AVEC LE CHEMIN CORRIGÉ")
    print("-" * 40)
    
    # Compter les images par classe
    total_images = 0
    class_distribution = {}
    
    for class_name in CLASS_NAMES:
        class_path = correct_path / class_name
        if class_path.exists():
            images = list(class_path.glob('**/*.png')) + list(class_path.glob('**/*.jpg'))
            class_distribution[class_name] = len(images)
            total_images += len(images)
            print(f"📁 {class_name}: {len(images)} images")
        else:
            class_distribution[class_name] = 0
            print(f"📁 {class_name}: 0 images (dossier inexistant)")
    
    print(f"\n📊 RÉSUMÉ AVEC CHEMIN CORRIGÉ:")
    print(f"• Total images: {total_images}")
    print(f"• Répartition: {class_distribution}")
    
    if total_images > 0:
        print(f"\n🎉 PROBLÈME RÉSOLU !")
        print(f"💡 Le problème était un chemin relatif incorrect")
        print(f"🔧 Solution: Utiliser le chemin '{corrected_data_dir}'")
        
        # Mettre à jour la variable globale pour la suite
        DATA_DIR_CORRECTED = corrected_data_dir
        print(f"📝 Variable DATA_DIR_CORRECTED créée: {DATA_DIR_CORRECTED}")
        
        # Test rapide du DataLoader avec le bon chemin
        if DATALOADER_AVAILABLE:
            print(f"\n🧪 TEST RAPIDE DU DATALOADER CORRIGÉ")
            print("-" * 40)
            
            try:
                from src.features.raf.data import DataLoader
                from src.features.raf.utils import get_config
                
                # Configuration avec le bon chemin
                config = get_config()
                config.data_dir = correct_path
                config.classes = CLASS_NAMES
                config.max_images_per_class = 5  # Test rapide
                
                loader_corrected = DataLoader(data_dir=correct_path, config=config)
                
                with tqdm(total=1, desc="Test DataLoader corrigé", colour='green') as pbar:
                    image_paths, labels, class_counts = loader_corrected.load_image_paths_and_labels()
                    pbar.update(1)
                
                print(f"✅ DataLoader avec chemin corrigé: {len(image_paths)} images trouvées")
                print(f"📊 Comptage par classe: {dict(class_counts)}")
                
                if len(image_paths) > 0:
                    print(f"🎯 DATALOADER FONCTIONNEL !")
                else:
                    print(f"⚠️ DataLoader toujours ne trouve pas d'images")
                    
            except Exception as e:
                print(f"❌ Erreur test DataLoader corrigé: {e}")
    
else:
    print(f"\n❌ AUCUN CHEMIN CORRECT TROUVÉ")
    print(f"💡 Suggestions:")
    print(f"  1. Vérifiez que le dataset COVID-19 est bien téléchargé")
    print(f"  2. Vérifiez la structure des dossiers:")
    print(f"     data/raw/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset/")
    print(f"       ├── COVID/")
    print(f"       ├── Lung_Opacity/")
    print(f"       ├── Normal/")
    print(f"       └── Viral Pneumonia/")
    print(f"  3. Relancez depuis le répertoire racine du projet")

print(f"\n🏁 Diagnostic terminé !")

🔧 CORRECTION AUTOMATIQUE DES CHEMINS
📍 Répertoire de travail actuel: /home/cepa/DST/projet_DS/DS_COVID/notebooks

🔍 Test des chemins possibles...


Test chemins:   0%|          | 0/4 [00:00<?, ?it/s]


📂 Test 1: ../data/raw/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset
   📍 Absolu: /home/cepa/DST/projet_DS/DS_COVID/data/raw/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset
   📂 Existe: True
   ✅ COVID: True
   ✅ Lung_Opacity: True
   ✅ Normal: True
   ✅ Viral Pneumonia: True
   🎯 CHEMIN CORRECT TROUVÉ !

✅ CHEMIN CORRIGÉ TROUVÉ !
📂 Chemin correct: ../data/raw/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset
📍 Chemin absolu: /home/cepa/DST/projet_DS/DS_COVID/data/raw/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset

🔄 Mise à jour de la configuration...

🧪 TEST AVEC LE CHEMIN CORRIGÉ
----------------------------------------
📁 COVID: 7232 images
📁 Lung_Opacity: 12024 images
📁 Normal: 20384 images
📁 Viral Pneumonia: 2690 images

📊 RÉSUMÉ AVEC CHEMIN CORRIGÉ:
• Total images: 42330
• Répartition: {'COVID': 7232, 'Lung_Opacity': 12024, 'Normal': 20384, 'Viral Pneumonia': 2690}

🎉 PROBLÈME RÉSOLU !
💡 Le problème était un chemin relatif incorrect
🔧 Solut

Test DataLoader corrigé:   0%|          | 0/1 [00:00<?, ?it/s]

🔍 Chargement des chemins d'images...
  COVID: 3616 images
  Lung_Opacity: 6012 images
  Normal: 10192 images
  Viral Pneumonia: 1345 images
📊 Total: 21165 images
✅ DataLoader avec chemin corrigé: 21165 images trouvées
📊 Comptage par classe: {'COVID': 3616, 'Lung_Opacity': 6012, 'Normal': 10192, 'Viral Pneumonia': 1345}
🎯 DATALOADER FONCTIONNEL !

🏁 Diagnostic terminé !
  Normal: 10192 images
  Viral Pneumonia: 1345 images
📊 Total: 21165 images
✅ DataLoader avec chemin corrigé: 21165 images trouvées
📊 Comptage par classe: {'COVID': 3616, 'Lung_Opacity': 6012, 'Normal': 10192, 'Viral Pneumonia': 1345}
🎯 DATALOADER FONCTIONNEL !

🏁 Diagnostic terminé !


In [ ]:
# Création de modèles simulés pour la démonstration
# En pratique, vous chargerez vos modèles pré-entraînés

print("🏗️ Création des modèles de démonstration...")

# Configuration depuis .env
EPOCHS = int(os.getenv('EPOCHS', 2))  # Epochs rapides pour démo
BATCH_SIZE = int(os.getenv('BATCH_SIZE', 32))
LEARNING_RATE = float(os.getenv('LEARNING_RATE', 0.001))

# Configuration Random Forest depuis .env
RF_N_ESTIMATORS = int(os.getenv('RF_N_ESTIMATORS', 50))
RF_MAX_DEPTH = int(os.getenv('RF_MAX_DEPTH', 10))

# Modèle CNN simulé (TensorFlow/Keras)
try:
    import tensorflow as tf
    from tensorflow import keras
    
    # CNN simple pour démonstration
    cnn_model = keras.Sequential([
        keras.layers.Conv2D(32, 3, activation='relu', input_shape=(*IMG_SIZE, 1)),
        keras.layers.MaxPooling2D(),
        keras.layers.Conv2D(64, 3, activation='relu'),
        keras.layers.MaxPooling2D(),
        keras.layers.Conv2D(128, 3, activation='relu'),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    
    cnn_model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print("✅ Modèle CNN créé")
    print(f"📊 Architecture: {len(cnn_model.layers)} couches")
    print(f"🎯 Learning rate: {LEARNING_RATE}")
    
    # Entraînement rapide pour avoir des poids
    print(f"🏋️ Entraînement rapide ({EPOCHS} époques)...")
    cnn_model.fit(X_images, y_true, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)
    print("✅ Modèle CNN entraîné")
    
    CNN_AVAILABLE = True
    
except ImportError:
    print("❌ TensorFlow non disponible - pas de modèle CNN")
    cnn_model = None
    CNN_AVAILABLE = False

# Modèle ML classique (Random Forest)
try:
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    
    # Création et entraînement du Random Forest avec config .env
    ml_model = RandomForestClassifier(
        n_estimators=RF_N_ESTIMATORS,
        max_depth=RF_MAX_DEPTH,
        random_state=RANDOM_SEED
    )
    
    # Entraînement
    ml_model.fit(X_flat, y_true)
    
    print("✅ Modèle Random Forest créé et entraîné")
    print(f"📊 Nombre d'arbres: {ml_model.n_estimators}")
    print(f"🌳 Profondeur max: {ml_model.max_depth}")
    
    ML_AVAILABLE = True
    
except ImportError:
    print("❌ Scikit-learn non disponible - pas de modèle ML")
    ml_model = None
    ML_AVAILABLE = False

print(f"\n📊 Résumé des modèles:")
print(f"• CNN disponible: {CNN_AVAILABLE}")
print(f"• ML disponible: {ML_AVAILABLE}")
print(f"• Configuration depuis .env: ✅")

## 🎯 Démonstration GradCAM

GradCAM (Gradient-weighted Class Activation Mapping) permet de visualiser les régions importantes pour la prédiction d'un CNN.

In [ ]:
if CNN_AVAILABLE and RAF_AVAILABLE:
    print("🎯 Démonstration GradCAM")
    print("=" * 40)
    
    # Initialisation de l'explainer GradCAM
    try:
        gradcam_explainer = GradCAMExplainer(cnn_model)
        print(f"✅ GradCAM explainer initialisé")
        print(f"🔍 Couche analysée: {gradcam_explainer.layer_name}")
        
        # Sélection d'une image pour l'analyse
        test_image = X_images[0]  # Première image
        true_class = y_true[0]
        
        print(f"\n🖼️ Analyse de l'image test")
        print(f"• Classe réelle: {CLASS_NAMES[true_class]}")
        print(f"• Shape: {test_image.shape}")
        
        # Génération de la carte GradCAM avec paramètres .env
        print(f"\n🔍 Génération GradCAM (alpha={GRADCAM_ALPHA}, colormap={GRADCAM_COLORMAP})...")
        gradcam_results = gradcam_explainer.generate_gradcam(
            test_image, 
            alpha=GRADCAM_ALPHA, 
            colormap=GRADCAM_COLORMAP
        )
        
        predicted_class = gradcam_results['predicted_class']
        confidence = gradcam_results['prediction_confidence']
        
        print(f"✅ GradCAM généré !")
        print(f"• Classe prédite: {CLASS_NAMES[predicted_class]}")
        print(f"• Confiance: {confidence:.2%}")
        
        # Seuils de confiance depuis .env
        confidence_high = float(os.getenv('CONFIDENCE_HIGH_THRESHOLD', 0.8))
        confidence_medium = float(os.getenv('CONFIDENCE_MEDIUM_THRESHOLD', 0.6))
        
        if confidence > confidence_high:
            print("🔥 Confiance élevée")
        elif confidence > confidence_medium:
            print("⚡ Confiance moyenne")
        else:
            print("⚠️ Confiance faible")
        
        # Visualisation des résultats
        fig = gradcam_explainer.plot_gradcam(
            gradcam_results,
            title=f"Analyse GradCAM - {CLASS_NAMES[predicted_class]}",
            class_names=CLASS_NAMES
        )
        plt.show()
        
        # Analyse quantitative
        from src.features.raf.interpretability.gradcam_explainer import (
            extract_gradcam_features,
            analyze_gradcam_regions
        )
        
        features = extract_gradcam_features(gradcam_results['heatmap'])
        regions = analyze_gradcam_regions(gradcam_results['heatmap'])
        
        print(f"\n📊 Analyse quantitative:")
        print(f"• Activation moyenne: {features['mean_activation']:.3f}")
        print(f"• Activation maximale: {features['max_activation']:.3f}")
        print(f"• Entropie: {features['entropy']:.3f}")
        print(f"• Concentration >75%: {features['concentration_75']:.1%}")
        print(f"• Région importante: {regions['importance_ratio']:.1%}")
        print(f"• Configuration depuis .env: ✅")
        
    except Exception as e:
        print(f"❌ Erreur GradCAM: {e}")
        
else:
    print("⚠️ GradCAM non disponible (CNN ou RAF manquant)")

In [ ]:
if CNN_AVAILABLE and RAF_AVAILABLE:
    print("🔄 Comparaison GradCAM vs GradCAM++")
    print("=" * 40)
    
    try:
        # Génération des deux méthodes
        test_image = X_images[1]  # Deuxième image
        
        gradcam_result = gradcam_explainer.generate_gradcam(test_image)
        gradcam_plus_result = gradcam_explainer.generate_gradcam_plus_plus(test_image)
        
        # Comparaison visuelle
        from src.features.raf.interpretability.gradcam_explainer import visualize_gradcam_comparison
        
        comparison_results = {
            'GradCAM': gradcam_result,
            'GradCAM++': gradcam_plus_result
        }
        
        fig = visualize_gradcam_comparison(
            comparison_results,
            title="Comparaison GradCAM vs GradCAM++",
            class_names=CLASS_NAMES
        )
        plt.show()
        
        # Comparaison quantitative
        features_gradcam = extract_gradcam_features(gradcam_result['heatmap'])
        features_gradcam_plus = extract_gradcam_features(gradcam_plus_result['heatmap'])
        
        comparison_df = pd.DataFrame({
            'Métrique': ['Activation moyenne', 'Activation max', 'Entropie', 'Concentration 75%'],
            'GradCAM': [
                features_gradcam['mean_activation'],
                features_gradcam['max_activation'],
                features_gradcam['entropy'],
                features_gradcam['concentration_75']
            ],
            'GradCAM++': [
                features_gradcam_plus['mean_activation'],
                features_gradcam_plus['max_activation'],
                features_gradcam_plus['entropy'],
                features_gradcam_plus['concentration_75']
            ]
        })
        
        print("\n📊 Comparaison quantitative:")
        print(comparison_df.to_string(index=False, float_format='%.3f'))
        
    except Exception as e:
        print(f"❌ Erreur comparaison: {e}")
        
else:
    print("⚠️ Comparaison GradCAM non disponible")

## 📊 Démonstration SHAP

SHAP (SHapley Additive exPlanations) permet d'expliquer les prédictions de tout type de modèle en quantifiant l'importance de chaque feature.

## 🍃 Démonstration LIME

LIME (Local Interpretable Model-agnostic Explanations) explique les prédictions en approximant localement le modèle avec un modèle interprétable.

In [ ]:
if CNN_AVAILABLE and RAF_AVAILABLE:
    print("🍃 Démonstration LIME")
    print("=" * 40)
    
    try:
        # Vérifier que LIME est disponible
        try:
            import lime
            print("✅ LIME disponible")
            LIME_AVAILABLE = True
        except ImportError:
            print("❌ LIME non disponible - installation nécessaire")
            print("!pip install lime")
            LIME_AVAILABLE = False
        
        if LIME_AVAILABLE:
            # Initialisation de l'explainer LIME
            def preprocess_for_lime(images):
                """Fonction de préprocessing pour LIME"""
                if len(images.shape) == 3:
                    images = np.expand_dims(images, axis=0)
                # Normalisation pour le modèle CNN
                return images.astype(np.float32) / 255.0
            
            lime_explainer = LIMEExplainer(cnn_model, preprocess_fn=preprocess_for_lime)
            print(f"✅ LIME explainer initialisé")
            
            # Sélection d'une image pour l'analyse LIME
            test_image = X_images[0]  # Première image
            true_class = y_true[0]
            
            print(f"\n🖼️ Analyse LIME de l'image test")
            print(f"• Classe réelle: {CLASS_NAMES[true_class]}")
            print(f"• Shape: {test_image.shape}")
            print(f"• Échantillons LIME: {LIME_NUM_SAMPLES}")
            
            # Génération de l'explication LIME
            print(f"\n🔍 Génération de l'explication LIME...")
            lime_results = lime_explainer.explain_image(
                test_image.squeeze(),  # LIME attend (H, W, C)
                top_labels=NUM_CLASSES,
                num_samples=LIME_NUM_SAMPLES,
                num_features=100
            )
            
            predicted_class = lime_results['predicted_class']
            confidence = lime_results['prediction_confidence']
            
            print(f"✅ Explication LIME générée !")
            print(f"• Classe prédite: {CLASS_NAMES[predicted_class]}")
            print(f"• Confiance: {confidence:.2%}")
            
            # Visualisation des résultats LIME
            fig = lime_explainer.plot_lime_explanation(
                lime_results,
                title=f"Analyse LIME - {CLASS_NAMES[predicted_class]}",
                class_names=CLASS_NAMES,
                num_features=10
            )
            plt.show()
            
            # Analyse quantitative LIME
            analysis = lime_explainer.analyze_lime_features(lime_results)
            
            print(f"\n📊 Analyse quantitative LIME:")
            print(f"• Nombre total de superpixels: {analysis['num_features']}")
            print(f"• Features positives: {analysis['positive_features']}")
            print(f"• Features négatives: {analysis['negative_features']}")
            print(f"• Ratio positif: {analysis['positive_ratio']:.1%}")
            print(f"• Poids total: {analysis['total_weight']:.3f}")
            
            # Top features
            print(f"\n🏆 Top 5 Features Positives:")
            for i, (feature_id, weight) in enumerate(analysis['top_positive_features'][:5]):
                print(f"  {i+1}. Superpixel {feature_id}: {weight:.3f}")
            
            if analysis['top_negative_features']:
                print(f"\n⚠️ Top 5 Features Négatives:")
                for i, (feature_id, weight) in enumerate(analysis['top_negative_features'][:5]):
                    print(f"  {i+1}. Superpixel {feature_id}: {weight:.3f}")
        
        else:
            print("⚠️ LIME non disponible pour cette démonstration")
            
    except Exception as e:
        print(f"❌ Erreur LIME: {e}")
        import traceback
        traceback.print_exc()
        
else:
    print("⚠️ LIME non disponible (CNN ou RAF manquant)")

In [ ]:
if CNN_AVAILABLE and RAF_AVAILABLE and 'lime_explainer' in locals():
    print("🔄 Comparaison LIME avec Multiple Images")
    print("=" * 40)
    
    try:
        # Sélection de plusieurs images pour comparaison
        n_compare = min(3, len(X_images))
        compare_images = [X_images[i].squeeze() for i in range(n_compare)]
        compare_labels = [f"{CLASS_NAMES[y_true[i]]} (réel)" for i in range(n_compare)]
        
        print(f"🔍 Comparaison LIME sur {n_compare} images...")
        
        # Comparaison LIME
        comparison_results = lime_explainer.compare_lime_explanations(
            compare_images,
            labels=compare_labels,
            num_samples=500  # Moins d'échantillons pour accélérer
        )
        
        print("✅ Comparaison LIME terminée !")
        
        # Visualisation de la comparaison
        fig = visualize_lime_comparison(comparison_results, class_names=CLASS_NAMES)
        if fig is not None:
            plt.show()
        
        # Analyse comparative des résultats
        analyses = comparison_results['analyses']
        valid_analyses = [a for a in analyses if a is not None]
        
        if valid_analyses:
            print(f"\n📊 Analyse Comparative LIME:")
            
            # Statistiques moyennes
            avg_positive = np.mean([a['positive_features'] for a in valid_analyses])
            avg_negative = np.mean([a['negative_features'] for a in valid_analyses])
            avg_ratio = np.mean([a['positive_ratio'] for a in valid_analyses])
            
            print(f"• Features positives (moyenne): {avg_positive:.1f}")
            print(f"• Features négatives (moyenne): {avg_negative:.1f}")
            print(f"• Ratio positif (moyenne): {avg_ratio:.1%}")
            
            # Comparaison par image
            for i, (analysis, label) in enumerate(zip(analyses, compare_labels)):
                if analysis:
                    print(f"\n{label}:")
                    print(f"  • Features: {analysis['positive_features']}(+) / {analysis['negative_features']}(-)")
                    print(f"  • Ratio positif: {analysis['positive_ratio']:.1%}")
        
    except Exception as e:
        print(f"❌ Erreur comparaison LIME: {e}")
        
else:
    print("⚠️ Comparaison LIME non disponible")

In [ ]:
if ML_AVAILABLE and RAF_AVAILABLE:
    print("📊 Démonstration SHAP")
    print("=" * 40)
    
    try:
        # Configuration SHAP depuis .env
        shap_model_type = os.getenv('SHAP_MODEL_TYPE', 'tree')
        
        # Initialisation de l'explainer SHAP
        shap_explainer = SHAPExplainer(ml_model, model_type=shap_model_type)
        print(f"✅ SHAP explainer initialisé (type: {shap_model_type})")
        
        # Préparation des données avec taille configurée dans .env
        X_background = X_flat[:SHAP_BACKGROUND_SIZE]  # Échantillons d'arrière-plan
        X_explain = X_flat[SHAP_BACKGROUND_SIZE:SHAP_BACKGROUND_SIZE+2]   # Échantillons à expliquer
        
        print(f"\n🔧 Configuration depuis .env:")
        print(f"• Données arrière-plan: {X_background.shape} (size={SHAP_BACKGROUND_SIZE})")
        print(f"• Données à expliquer: {X_explain.shape}")
        print(f"• Max évaluations: {SHAP_MAX_EVALS}")
        
        # Ajustement de l'explainer
        print("\n🔍 Ajustement de l'explainer...")
        shap_explainer.fit_explainer(X_background)
        
        # Calcul des valeurs SHAP
        print("📊 Calcul des valeurs SHAP...")
        shap_values = shap_explainer.explain(X_explain, max_evals=SHAP_MAX_EVALS)
        
        print(f"✅ Valeurs SHAP calculées !")
        print(f"• Shape: {shap_values.shape if not isinstance(shap_values, list) else [v.shape for v in shap_values]}")
        
        # Visualisation waterfall pour le premier échantillon
        print("\n🌊 Graphique Waterfall...")
        try:
            # Sélection des features les plus importantes pour la visualisation
            if isinstance(shap_values, list):
                sample_shap = shap_values[0][0]  # Premier échantillon, première classe
            else:
                sample_shap = shap_values[0]     # Premier échantillon
            
            # Top 20 features par importance absolue
            top_indices = np.argsort(np.abs(sample_shap))[-20:]
            feature_names = [f'Pixel_{i}' for i in top_indices]
            
            fig = shap_explainer.plot_waterfall(
                instance_idx=0,
                feature_names=feature_names,
                show=True
            )
            
        except Exception as e:
            print(f"⚠️ Waterfall non disponible: {e}")
            # Graphique alternatif
            fig, ax = plt.subplots(figsize=(12, 6))
            
            if isinstance(shap_values, list):
                values = shap_values[0][0][:50]  # Top 50 pixels
            else:
                values = shap_values[0][:50]
            
            colors = ['red' if v > 0 else 'blue' for v in values]
            bars = ax.bar(range(len(values)), values, color=colors, alpha=0.7)
            ax.set_xlabel('Pixels')
            ax.set_ylabel('Valeur SHAP')
            ax.set_title('Contribution des Pixels (SHAP) - Top 50')
            ax.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()
        
        # Importance globale des features
        print("\n📈 Importance globale des features...")
        importance_df = shap_explainer.get_feature_importance()
        
        # Visualisation de l'importance
        fig, ax = plt.subplots(figsize=(12, 8))
        top_features = importance_df.head(20)
        
        bars = ax.barh(range(len(top_features)), top_features['importance'], alpha=0.7)
        ax.set_yticks(range(len(top_features)))
        ax.set_yticklabels([f'Pixel_{i}' for i in range(len(top_features))])
        ax.set_xlabel('Importance SHAP moyenne')
        ax.set_title('Top 20 Pixels - Importance SHAP')
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        # Statistiques SHAP
        if isinstance(shap_values, list):
            stats_values = shap_values[0]
        else:
            stats_values = shap_values
            
        print(f"\n📊 Statistiques SHAP:")
        print(f"• Nombre d'échantillons: {stats_values.shape[0]}")
        print(f"• Nombre de features: {stats_values.shape[1]}")
        print(f"• Valeur moyenne: {np.mean(stats_values):.4f}")
        print(f"• Valeur max: {np.max(stats_values):.4f}")
        print(f"• Valeur min: {np.min(stats_values):.4f}")
        print(f"• Écart-type: {np.std(stats_values):.4f}")
        print(f"• Configuration depuis .env: ✅")
        
    except Exception as e:
        print(f"❌ Erreur SHAP: {e}")
        import traceback
        traceback.print_exc()
        
else:
    print("⚠️ SHAP non disponible (ML ou RAF manquant)")

## ⚖️ Analyse Comparative SHAP vs GradCAM

Comparaison des explications fournies par les deux méthodes sur la même image.

In [ ]:
if CNN_AVAILABLE and ML_AVAILABLE and RAF_AVAILABLE:
    print("⚖️ Analyse Comparative SHAP vs GradCAM")
    print("=" * 50)
    
    try:
        # Initialisation de l'analyseur intégré
        analyzer = InterpretabilityAnalyzer(
            cnn_model=cnn_model,
            ml_model=ml_model
        )
        
        # Sélection d'une image pour l'analyse comparative
        test_idx = 2
        test_image_cnn = X_images[test_idx]
        test_image_ml = X_flat[test_idx]
        true_class = y_true[test_idx]
        
        print(f"\n🖼️ Image test sélectionnée:")
        print(f"• Index: {test_idx}")
        print(f"• Classe réelle: {CLASS_NAMES[true_class]}")
        print(f"• Shape CNN: {test_image_cnn.shape}")
        print(f"• Shape ML: {test_image_ml.shape}")
        
        # Analyse comparative
        print("\n🔍 Lancement de l'analyse comparative...")
        comparison_results = analyzer.compare_predictions(
            img=test_image_cnn,
            X_flat=test_image_ml.reshape(1, -1),
            X_background=X_flat[:3],  # Données d'arrière-plan
            class_names=CLASS_NAMES
        )
        
        print("✅ Analyse comparative terminée !")
        
        # Création du tableau de bord intégré
        print("\n📊 Génération du tableau de bord...")
        dashboard_fig = create_interpretability_dashboard(
            analyzer=analyzer,
            img=test_image_cnn,
            X_flat=test_image_ml,
            X_background=X_flat[:3],
            class_names=CLASS_NAMES
        )
        plt.show()
        
        # Rapport textuel de comparaison
        if 'comparison_report' in comparison_results:
            report = comparison_results['comparison_report']
            print(f"\n📋 Rapport de Comparaison:")
            print(f"• Confiance CNN: {report.get('cnn_confidence', 'N/A')}")
            print(f"• Classe prédite CNN: {report.get('cnn_predicted_class', 'N/A')}")
            print(f"• Résumé: {report.get('summary', 'N/A')}")
        
        # Cohérence des explications
        print(f"\n🎯 Analyse de Cohérence:")
        
        if 'cnn' in comparison_results and 'ml' in comparison_results:
            cnn_confidence = comparison_results['cnn']['gradcam']['prediction_confidence']
            cnn_class = comparison_results['cnn']['gradcam']['predicted_class']
            
            print(f"• GradCAM - Classe: {CLASS_NAMES[cnn_class]}, Confiance: {cnn_confidence:.2%}")
            print(f"• Classe réelle: {CLASS_NAMES[true_class]}")
            
            if cnn_class == true_class:
                print("✅ Prédiction correcte !")
            else:
                print("❌ Prédiction incorrecte")
            
            if cnn_confidence > 0.8:
                print("🔥 Confiance élevée")
            elif cnn_confidence > 0.6:
                print("⚡ Confiance moyenne")
            else:
                print("⚠️ Confiance faible")
        
    except Exception as e:
        print(f"❌ Erreur analyse comparative: {e}")
        import traceback
        traceback.print_exc()
        
else:
    print("⚠️ Analyse comparative non disponible")
    print(f"• CNN: {CNN_AVAILABLE}")
    print(f"• ML: {ML_AVAILABLE}")
    print(f"• RAF: {RAF_AVAILABLE}")

## 📈 Analyse Batch et Rapport

Génération d'un rapport d'interprétabilité sur plusieurs échantillons.

In [ ]:
if CNN_AVAILABLE and RAF_AVAILABLE:
    print("📈 Génération de Rapport Batch")
    print("=" * 40)
    
    try:
        # Sélection d'un batch d'images
        batch_size = 5
        batch_images = X_images[:batch_size]
        batch_labels = y_true[:batch_size]
        
        print(f"\n📊 Configuration du batch:")
        print(f"• Nombre d'images: {batch_size}")
        print(f"• Classes: {[CLASS_NAMES[label] for label in batch_labels]}")
        
        # Génération du rapport
        print("\n🔍 Génération du rapport...")
        
        analyzer = InterpretabilityAnalyzer(cnn_model=cnn_model)
        
        report_df = generate_interpretability_report(
            analyzer=analyzer,
            images=list(batch_images),
            X_background=X_flat[:3],
            class_names=CLASS_NAMES,
            sample_names=[f"Sample_{i+1}" for i in range(batch_size)]
        )
        
        print("✅ Rapport généré !")
        
        # Affichage du rapport
        print("\n📋 Rapport d'Interprétabilité:")
        print(report_df.to_string(index=False, float_format='%.3f'))
        
        # Analyse statistique du batch
        if not report_df.empty:
            print(f"\n📊 Statistiques du Batch:")
            print(f"• Confiance moyenne: {report_df['confidence'].mean():.2%}")
            print(f"• Confiance min: {report_df['confidence'].min():.2%}")
            print(f"• Confiance max: {report_df['confidence'].max():.2%}")
            
            # Distribution des classes prédites
            class_dist = report_df['predicted_class_name'].value_counts() if 'predicted_class_name' in report_df.columns else report_df['predicted_class'].value_counts()
            print(f"\n🏷️ Distribution des prédictions:")
            for class_name, count in class_dist.items():
                print(f"• {class_name}: {count} ({count/len(report_df)*100:.1f}%)")
            
            # Visualisation de la distribution des confiances
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
            
            # Histogramme des confiances
            ax1.hist(report_df['confidence'], bins=10, alpha=0.7, edgecolor='black')
            ax1.set_xlabel('Confiance')
            ax1.set_ylabel('Nombre d\'images')
            ax1.set_title('Distribution des Confiances')
            ax1.grid(True, alpha=0.3)
            
            # Graphique des activations moyennes
            if 'gradcam_mean_activation' in report_df.columns:
                ax2.scatter(report_df['confidence'], report_df['gradcam_mean_activation'], alpha=0.7)
                ax2.set_xlabel('Confiance')
                ax2.set_ylabel('Activation GradCAM Moyenne')
                ax2.set_title('Confiance vs Activation GradCAM')
                ax2.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
        
        # Résumé visuel avec les meilleures prédictions
        print("\n🖼️ Résumé visuel des meilleures prédictions...")
        
        # Génération des résultats GradCAM pour visualisation
        gradcam_results = []
        for img in batch_images:
            try:
                result = gradcam_explainer.generate_gradcam(img)
                gradcam_results.append(result)
            except:
                continue
        
        if gradcam_results:
            from src.features.raf.interpretability.utils import plot_interpretability_summary
            
            summary_fig = plot_interpretability_summary(
                gradcam_results[:4],  # Max 4 échantillons
                class_names=CLASS_NAMES
            )
            plt.show()
        
    except Exception as e:
        print(f"❌ Erreur rapport batch: {e}")
        import traceback
        traceback.print_exc()
        
else:
    print("⚠️ Rapport batch non disponible")

## 🏥 Application Médicale - Cas d'Usage COVID-19

Démonstration d'un cas d'usage réaliste pour le diagnostic médical.

In [ ]:
print("🏥 Cas d'Usage Médical - Diagnostic COVID-19")
print("=" * 50)

# Simulation d'un cas médical réaliste
print("\n👨‍⚕️ Scenario: Diagnostic d'une radiographie pulmonaire")
print("" * 50)

# Informations patient (simulées)
patient_info = {
    'id': 'P001',
    'age': 65,
    'sexe': 'M',
    'symptomes': ['Toux', 'Fièvre', 'Difficultés respiratoires'],
    'antecedents': ['Hypertension'],
    'date_exam': '2024-10-16'
}

print(f"📋 Informations Patient:")
for key, value in patient_info.items():
    print(f"• {key.capitalize()}: {value}")

# Sélection d'une image "suspecte"
suspect_image = X_images[0]
print(f"\n🖼️ Radiographie analysée: {suspect_image.shape}")

# Affichage de l'image
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.imshow(suspect_image.squeeze(), cmap='gray')
ax.set_title(f'Radiographie Patient {patient_info["id"]}', fontsize=14)
ax.axis('off')
plt.show()

if CNN_AVAILABLE and RAF_AVAILABLE:
    print("\n🔍 Analyse IA de la radiographie...")
    
    try:
        # Analyse GradCAM
        gradcam_result = gradcam_explainer.generate_gradcam(suspect_image)
        
        predicted_class = gradcam_result['predicted_class']
        confidence = gradcam_result['prediction_confidence']
        predicted_disease = CLASS_NAMES[predicted_class]
        
        print(f"\n🎯 Résultats de l'Analyse IA:")
        print(f"• Diagnostic suggéré: {predicted_disease}")
        print(f"• Confiance du modèle: {confidence:.1%}")
        
        # Interprétation médicale
        if predicted_disease == 'COVID':
            print(f"⚠️ ALERTE: Suspicion COVID-19 détectée")
            print(f"📋 Recommandations:")
            print(f"   • Test PCR recommandé")
            print(f"   • Isolement préventif")
            print(f"   • Surveillance des symptômes")
        elif predicted_disease == 'Normal':
            print(f"✅ Radiographie apparemment normale")
            print(f"📋 Recommandations:")
            print(f"   • Surveillance clinique")
            print(f"   • Suivi si symptômes persistent")
        else:
            print(f"⚠️ Anomalie détectée: {predicted_disease}")
            print(f"📋 Recommandations:")
            print(f"   • Évaluation spécialisée recommandée")
            print(f"   • Tests complémentaires")
        
        # Visualisation avec zones d'intérêt
        print(f"\n🔍 Zones d'attention identifiées par l'IA:")
        
        fig = gradcam_explainer.plot_gradcam(
            gradcam_result,
            title=f"Analyse IA - Patient {patient_info['id']} - {predicted_disease}",
            class_names=CLASS_NAMES
        )
        plt.show()
        
        # Analyse quantitative pour le rapport médical
        features = extract_gradcam_features(gradcam_result['heatmap'])
        regions = analyze_gradcam_regions(gradcam_result['heatmap'])
        
        print(f"\n📊 Analyse Quantitative (pour le dossier médical):")
        print(f"• Zone d'attention: {regions['importance_ratio']:.1%} de l'image")
        print(f"• Intensité moyenne des zones suspectes: {features['mean_activation']:.3f}")
        print(f"• Score de confiance technique: {confidence:.3f}")
        
        # Niveau de confiance pour le médecin
        if confidence > 0.9:
            confidence_level = "Très élevée"
            recommendation = "Diagnostic IA très fiable"
        elif confidence > 0.75:
            confidence_level = "Élevée"
            recommendation = "Diagnostic IA fiable, confirmation clinique recommandée"
        elif confidence > 0.6:
            confidence_level = "Modérée"
            recommendation = "Diagnostic IA incertain, expertise médicale nécessaire"
        else:
            confidence_level = "Faible"
            recommendation = "Diagnostic IA peu fiable, réévaluation recommandée"
        
        print(f"\n👨‍⚕️ Interprétation pour le Médecin:")
        print(f"• Niveau de confiance: {confidence_level}")
        print(f"• Recommandation: {recommendation}")
        
        # Génération d'un rapport médical structuré
        medical_report = f"""
RAPPORT D'ANALYSE IA - RADIOGRAPHIE PULMONAIRE
=============================================

Patient: {patient_info['id']}
Date: {patient_info['date_exam']}
Âge: {patient_info['age']} ans, Sexe: {patient_info['sexe']}

RÉSULTATS DE L'ANALYSE IA:
• Diagnostic suggéré: {predicted_disease}
• Confiance du modèle: {confidence:.1%}
• Niveau de confiance: {confidence_level}

ANALYSE QUANTITATIVE:
• Zone d'attention: {regions['importance_ratio']:.1%} de l'image
• Intensité des anomalies: {features['mean_activation']:.3f}
• Score de concentration: {features['concentration_75']:.1%}

RECOMMANDATIONS CLINIQUES:
{recommendation}

NOTES:
• Cette analyse IA est un outil d'aide au diagnostic
• L'expertise médicale reste indispensable
• En cas de doute, privilégier l'évaluation clinique

Généré automatiquement le {patient_info['date_exam']}
        """
        
        print(medical_report)
        
    except Exception as e:
        print(f"❌ Erreur analyse médicale: {e}")
        print(f"⚠️ En situation réelle, consulter un radiologue")
        
else:
    print("\n⚠️ Analyse IA non disponible")
    print("En situation réelle, la radiographie serait analysée par un médecin")

print(f"\n🏥 Fin du cas médical - Patient {patient_info['id']}")

## 📝 Conclusion et Bonnes Pratiques

### 🎯 Résumé des Techniques

1. **GradCAM** : Visualisation des zones importantes pour les CNN
   - ✅ Intuitive et visuelle
   - ✅ Spécifique aux réseaux convolutionnels
   - ❌ Limitée aux modèles avec couches convolutionnelles

2. **SHAP** : Quantification de l'importance des features
   - ✅ Universel (tous types de modèles)
   - ✅ Fondements théoriques solides
   - ❌ Plus lent sur de grandes données

3. **LIME** : Explication par approximation locale
   - ✅ Agnostique au modèle
   - ✅ Explications intuitives par superpixels
   - ✅ Fonctionne bien sur les images
   - ❌ Peut être instable selon l'échantillonnage

### 🏥 Applications Médicales

- **Aide au diagnostic** : Identifier les zones suspectes
- **Formation** : Comprendre les critères du modèle
- **Validation** : Vérifier la cohérence des prédictions
- **Confiance** : Évaluer la fiabilité des résultats
- **Traçabilité** : Documenter les décisions automatisées

### ⚖️ Bonnes Pratiques

1. **Combinaison des méthodes** : Utiliser SHAP, GradCAM ET LIME
2. **Validation humaine** : Toujours confirmer avec un expert
3. **Seuils de confiance** : Définir des niveaux d'alerte
4. **Documentation** : Tracer les décisions du modèle
5. **Données réelles** : Tester sur de vraies données médicales

### 🔧 Framework RAF

Le framework RAF facilite l'intégration de l'interprétabilité :
- Modules prêts à l'emploi (SHAP, GradCAM, LIME)
- Interface unifiée avec DataLoader intégré
- Visualisations automatiques
- Rapports structurés
- Configuration via fichiers .env

### 🚀 Prochaines Étapes

1. Intégrer vos vrais modèles entraînés
2. Tester sur de vraies données médicales COVID-19
3. Valider avec des experts médicaux
4. Déployer dans l'interface Streamlit
5. Automatiser les rapports d'interprétabilité
6. Comparer les résultats SHAP, GradCAM et LIME

### 📊 Recommandations d'Usage

- **GradCAM** : Pour visualiser rapidement les zones d'attention
- **LIME** : Pour expliquer des cas spécifiques aux médecins
- **SHAP** : Pour des analyses quantitatives et comparatives
- **Combinaison** : Pour une validation croisée des explications

In [ ]:
print("🎉 NOTEBOOK D'INTERPRÉTABILITÉ TERMINÉ")
print("=" * 50)

summary_stats = {
    'Modules testés': {
        'SHAP': 'Disponible' if RAF_AVAILABLE else 'Non disponible',
        'GradCAM': 'Disponible' if RAF_AVAILABLE else 'Non disponible',
        'LIME': 'Disponible' if RAF_AVAILABLE else 'Non disponible',
        'Framework RAF': 'Disponible' if RAF_AVAILABLE else 'Non disponible'
    },
    'Données': {
        'DataLoader RAF': 'Disponible' if DATALOADER_AVAILABLE else 'Non disponible',
        'Données réelles': 'Chargées' if 'REAL_DATA_LOADED' in locals() and REAL_DATA_LOADED else 'Simulées',
        'Dataset COVID-19': 'Utilisé' if 'REAL_DATA_LOADED' in locals() and REAL_DATA_LOADED else 'Non utilisé'
    },
    'Modèles utilisés': {
        'CNN (TensorFlow)': 'Créé' if CNN_AVAILABLE else 'Non disponible',
        'ML (Random Forest)': 'Créé' if ML_AVAILABLE else 'Non disponible'
    },
    'Analyses réalisées': {
        'GradCAM simple': '✅' if CNN_AVAILABLE and RAF_AVAILABLE else '❌',
        'GradCAM++': '✅' if CNN_AVAILABLE and RAF_AVAILABLE else '❌',
        'LIME Image': '✅' if CNN_AVAILABLE and RAF_AVAILABLE else '❌',
        'LIME Comparaison': '✅' if CNN_AVAILABLE and RAF_AVAILABLE else '❌',
        'SHAP Tree': '✅' if ML_AVAILABLE and RAF_AVAILABLE else '❌',
        'Analyse comparative': '✅' if CNN_AVAILABLE and ML_AVAILABLE and RAF_AVAILABLE else '❌',
        'Rapport batch': '✅' if CNN_AVAILABLE and RAF_AVAILABLE else '❌',
        'Cas médical': '✅' if CNN_AVAILABLE and RAF_AVAILABLE else '❌'
    }
}

for category, items in summary_stats.items():
    print(f"\n{category}:")
    for item, status in items.items():
        print(f"  • {item}: {status}")

print(f"\n📚 Ressources:")
print(f"  • Framework RAF: src/features/raf/interpretability/")
print(f"  • DataLoader RAF: src/features/raf/data/")
print(f"  • Interface Streamlit: src/streamlit/pages/03_Interpretability.py")
print(f"  • Documentation SHAP: https://shap.readthedocs.io/")
print(f"  • Paper GradCAM: https://arxiv.org/abs/1610.02391")
print(f"  • Paper LIME: https://arxiv.org/abs/1602.04938")

print(f"\n🚀 Prochaines étapes:")
print(f"  1. Installer LIME si nécessaire: pip install lime")
print(f"  2. Charger vos modèles réels entraînés")
print(f"  3. Tester sur vraies données COVID-19")
print(f"  4. Valider avec experts médicaux")
print(f"  5. Comparer SHAP, GradCAM et LIME")
print(f"  6. Intégrer dans pipeline de production")

print(f"\n✨ L'interprétabilité est essentielle pour l'IA médicale !")
print(f"🔍 SHAP + GradCAM + LIME = Triangle d'or de l'explicabilité")
print(f"🏥 Continuez à explorer et à valider vos modèles.")